In [1]:
###import statements

import pandas as pd
import numpy as np
import cv2
import itertools
import matplotlib.pyplot as plt

<br>

Part 1

<br>

In [18]:
# Load the image
image = cv2.imread('house1.png')

# Convert the image to grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Apply edge detection
edges = cv2.Canny(gray, 50, 150, apertureSize=3)



In [19]:

#### to get all the lines in the image and
## calculate the slope
def getParallelLines(img, lines):
    #https://hackthedeveloper.com/line-detection-opencv-python/
    i=0
    for line in lines:
        rho, theta = line[0]
        a = np.cos(theta)
        b = np.sin(theta)
        x0 = a*rho
        y0 = b*rho
        x1 = int(x0 + 1000*(-b))
        y1 = int(y0 + 1000*(a))
        x2 = int(x0 - 1000*(-b))
        y2 = int(y0 - 1000*(a))

        
        ##to calculate the slope, so it can be used to get which lines are parallel
        ##to remove the devision by 0 error that shows up
        if (x2- x1) == 0:
#             slopeDictionary.update({key: 0}) 
#             linesDictionary.append({(rho, theta)})  
            continue
        else:
            key = f'key{line+1}'
            key2 = 'slope'
            rho4Dict = 'rho'
            theta4Dict = 'theta'
            slope = (y2 - y1) / (x2 - x1)
            ###making a dictionary for the slope, rho and theta, so it can be used later to draw the lines
            slopeDictionary.update({key: {key2:slope, rho4Dict: rho, theta4Dict: theta} })

        ###draws the line in the image where there are lines detected
        cv2.line(img, (x1, y1), (x2, y2), (0, 0, 255), 1)

In [20]:
# Apply Hough line detection

lines = cv2.HoughLines(edges, rho=4.5, theta=np.pi/180, threshold=100)
getParallelLines(img, lines)

# Show result
# img = cv2.resize(img, dsize=(600, 600))
cv2.imshow('image', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
print(f"the size of the dictionary is {len(slopeDictionary)}")
slopeDictionary.items()

In [ ]:
##making list so that it can store the individual variables needed to make the lines when identifying the pairs at a later p
slopeValue=[]
thetaValue=[]
rhoValue =[]
keysV = [] ###store all the key values

for keys in itertools.product(slopeDictionary.keys()):
    keysV.append(keys)
    value = slopeDictionary
    for key in keys:
        value = value[key]
#         print(value)
    thetaValue.append(value.get('theta'))
    slopeValue.append(value.get('slope'))
    rhoValue.append(value.get('rho'))
print(f"Keys: {keysV}, \n\nTheta: {thetaValue},\n\n Rho: {rhoValue},\n\n slope: {slopeValue}")


In [ ]:
df = pd.DataFrame()

In [ ]:
##to get the pairs of parallel lines in the image

keysIndex = []   ###stores all the sets of keys that are parallel matching pairs

##to find the slopes which are the same
counter = 0
storeIndex = []
for i in range(len(slopeValue)):
        for j in range(i, len(slopeValue)):
            ##checks if the values are the same
            ##wherever the values are the same and the keys(index) not the sameit means the lines are parallel
            if slopeValue[i] == slopeValue[j] and i != j:
                counter+=1
                print(f"Parallel lines found: ({slopeValue[i]}, {slopeValue[j]}) at index {i} and {j}")
                storeIndex.append(i)
                storeIndex.append(j)
                keysIndex.append((keysV[i], keysV[j]))
        
                ###added the values into the dataframe
                df.loc[i, 'rho'] = rhoValue[i]
                df.loc[i, 'theta'] = thetaValue[i]
                df.loc[i, 'slope'] = slopeValue[i]
                df.loc[i, 'key'] = keysV[i]
                df.loc[j, 'rho'] = rhoValue[j]
                df.loc[j, 'theta'] = thetaValue[j]
                df.loc[j, 'slope'] = slopeValue[j]
                df.loc[j, 'key'] = keysV[j]
                

        

In [ ]:
###contains all the lines needed to find the points for the intersection
storeIndex = storeIndex[2],storeIndex[3], storeIndex[5],storeIndex[7],storeIndex[9], storeIndex[10],storeIndex[13],storeIndex[15], storeIndex[21]

In [22]:
##the intersection points of our parallel set chosen
intersection_points = np.unique(intersection_points, axis=0)
intersection_points

array([[-4.6335507e+02,  1.0764628e+01],
       [-4.5167630e+02,  1.1376686e+01],
       [-4.2063522e+02,  1.3003480e+01],
       [-3.2266141e+02,  9.1994164e+01],
       [-1.9020645e+02, -1.2315752e+03],
       [-1.6331075e+02, -1.1917008e+03],
       [-1.2503773e+02, -1.1349586e+03],
       [-1.0941797e+02,  2.1511034e+02],
       [-5.0715313e+01,  2.6192499e+02],
       [-3.7680084e+01,  2.6169745e+02],
       [-2.8989450e+01,  2.6154578e+02],
       [ 1.6716206e+02, -3.7375208e+02],
       [ 1.8565991e+02,  5.5563708e+02],
       [ 1.8978897e+02,  5.4213147e+02],
       [ 2.2313907e+02,  4.3304825e+02],
       [ 2.2364232e+02,  4.3140222e+02],
       [ 2.2987885e+02,  4.1100345e+02],
       [ 2.7720667e+02,  2.5620111e+02],
       [ 2.8500000e+02, -5.2705292e+02],
       [ 2.8500000e+02, -8.2093071e+01],
       [ 2.8500000e+02, -6.6773750e+01],
       [ 2.8500000e+02, -5.5398407e+01],
       [ 2.8500000e+02,  4.9984245e+01],
       [ 2.8500000e+02,  2.3071027e+02],
       [ 2.85000

<br>

Part 2

<br>

In [11]:
###add one to each intersection point
targetPoints = [(i, j,0, 1) for i, j in intersection_points]

In [ ]:
# #first and second row of the matrices A_i
# a = []
# b = []
# n = len(intersection_points)
# for i in range( 1, n):
#     a.append([0, 0, 0, 0,  -targetPoints[i][3]*intersection_points[i], targetPoints[i][2]*intersection_points[i]])
    
#     b.append([np.float32(targetPoints[i][3])*intersection_points[i], 0, 0, 0, 0,  np.float32(-targetPoints[i][1])*intersection_points[i]])
